In [4]:
import os
from semeval2016_task13.semeval_taxonomy import *
from scoring_program.utils import *

In [3]:
gs_path = "./semeval2016_task13/data/gs_taxo/EN/food_wordnet_en.taxo" 

In [4]:
gs_path = "./semeval2016_task13/data/gs_taxo/EN/food_wordnet_en.taxo" 
tax = SemEvalTaxonomy(gs_path, use_underscore=False)

In [5]:
pred_path = "./baselines/predictions/semeval/predicted_TAXI_science_en_only_orphan_hch.tsv"

In [6]:
predictions = read_dataset(pred_path)

In [7]:
final_results = {}

accuracy = 0

for word, variants in predictions.items():
    true_hypernym = tax.get_hypernym(word)[0].replace("_", " ")
    
    for variant in variants:
        if variant in word:
            final_results[word] = variant
            if variant != true_hypernym:
                print(word,"|",  true_hypernym, "|", variant)
                print(variants)
                print(true_hypernym == final_results[word])
                print("====")
    if word not in final_results:
        final_results[word] = variants[0]
        
    if true_hypernym == final_results[word]:
        accuracy += 1
#     else:
#         print(word,"|",  true_hypernym)
#         if 
#         print(variants)
# #     if (tax.get_hypernym(word)[0].replace("_", " ") in variants) and (true_hypernym != final_results[word]):
# #         print(tax.get_hypernym(word)[0].replace("_", " ") in variants)
# #         print(word,"|",  true_hypernym)
#     print("===")

NetworkXError: The node os is not in the digraph.

In [15]:
accuracy

31

In [16]:
final_results

{'os': 'history of religion',
 'german language and literature': 'linguistics',
 'american material culture': 'history',
 'paleobiology': 'ecology',
 'elementary particles and fields and string theory': 'physics',
 'plant biology': 'molecular biology',
 'biomechanics': 'physiology',
 'spanish literature': 'history',
 'cognition and perception': 'mathematics',
 'industrial and organizational psychology': 'physics',
 'criminology': 'psychology',
 'physical processes': 'engineering',
 'atomic, molecular and optical physics': 'physics',
 'mechanics of materials': 'history of religion',
 'astronomy and astrophysics': 'physics',
 'bioimaging and biomedical optics': 'science',
 'asian art and architecture history': 'history',
 'medicine and health': 'public health',
 'growth and development': 'mathematics',
 'photonics': 'physics',
 'american popular culture': 'history',
 'heat transfer, combustion': 'engineering',
 'aeronautical vehicles': 'engineering',
 'process control and systems': 'dyna

In [17]:
len(predictions)

158

In [1]:
import os
from semeval2016_task13.semeval_taxonomy import *
from scoring_program.utils import *

In [26]:
import torch
from torch import nn
import torch.nn.functional as F
import os
import numpy as np

import torchtext
import torchtext.datasets
import torchtext.data
from collections import Counter
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from torch.autograd import Variable
from tqdm import tqdm

import fasttext
import torch
from torch import nn
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset
from torch.nn.utils.rnn import pad_sequence

In [3]:
gs_path = "./semeval2016_task13/data/gs_taxo/EN/food_wordnet_en.taxo" 
tax = SemEvalTaxonomy(gs_path, use_underscore=False)

In [4]:
with open('./semeval2016_task13/data/gs_taxo/EN/food_wordnet_en.taxo', 'r') as f:
    text = f.read()

In [5]:
chars = tuple(set(text.replace("\n", " ").replace("\t", " ")))
int2char = dict(enumerate(chars))
char2int = {ch: ii for ii, ch in int2char.items()}

In [6]:
pairs = set()
all_ = set()
counter = 0
for line in text.split("\n"):
    if line:
        _, word, hypernym = line.split("\t")
        pairs.add((word, hypernym))
        all_.add(word)
        all_.add(hypernym)
        counter += 1

In [7]:
print(counter)

1576


In [8]:
from random import choice
nodes = list(tax.get_nodes())

def get_random(word, stop_word):
    random_node = choice(nodes)
    while random_node == stop_word or random_node == word:
        random_node = choice(nodes)
    return random_node

In [9]:
false_hypernyms = dict()
for word, hypernym in pairs:
    false_hypernyms[word] = get_random(word, hypernym)
    false_hypernyms[hypernym] = get_random(word, hypernym)

In [10]:
for word, hypernym in pairs:
    print([char2int[i] for i in word], [char2int[i] for i in hypernym], 1)
    break

[29, 23, 7, 37, 10, 27, 25, 11, 34, 14, 10] [25, 11, 34, 14, 10] 1


In [11]:
for word, hypernym in false_hypernyms.items():
    print([char2int[i] for i in word], [char2int[i] for i in hypernym], 0)
    break

[29, 23, 7, 37, 10, 27, 25, 11, 34, 14, 10] [26, 10, 32, 0, 7, 14, 14, 23, 7, 8] 0


In [63]:
pairs

{('white sauce', 'sauce'),
 ('mash', 'feed'),
 ('cocktail', 'mixed drink'),
 ('cracked wheat bread', 'bread'),
 ('french omelet', 'firm omelet'),
 ('chowder', 'soup'),
 ('white rice', 'rice'),
 ('compote', 'dessert'),
 ('self rising flour', 'mix'),
 ('corn muffin', 'muffin'),
 ('truffle', 'candy'),
 ('coq au vin', 'dish'),
 ('metheglin', 'mead'),
 ('pork and veal goulash', 'goulash'),
 ('brandy sling', 'sling'),
 ('madeira', 'fortified wine'),
 ('soul food', 'food'),
 ('schnitzel', 'dish'),
 ('coca cola', 'cola'),
 ('bonbon', 'candy'),
 ('irish', 'whiskey'),
 ('tuna fish salad', 'salad'),
 ('mix', 'concoction'),
 ('bread dough', 'dough'),
 ('garlic chive', 'flavorer'),
 ('chili vinegar', 'vinegar'),
 ('stuffed tomato', 'dish'),
 ('knish', 'turnover'),
 ('water', 'food'),
 ('high tea', 'dinner'),
 ('salmi', 'ragout'),
 ('egg', 'foodstuff'),
 ('lutefisk', 'dish'),
 ('home brew', 'alcohol'),
 ('manhattan clam chowder', 'clam chowder'),
 ('crab louis', 'salad'),
 ('malt', 'grain'),
 ('coqu

In [12]:
ftmodel = fasttext.load_model("baselines/models/cc.ru.300.bin")

In [28]:
def get_data_vectors(multi_word):
    words = multi_word.replace("_", " ").split()
    return np.sum(get_vectors(words), axis=0)/len(words)

def get_vectors(words):
    vectors = np.zeros((len(words), 300))
    for i, word in enumerate(words):
            vectors[i, :] = ftmodel[word]
    return vectors

In [29]:
class CustomDataset(Dataset):
    def __init__(self, pairs, false_hypernyms, fmodel):
        self.fasttext = fmodel
        self.dataset = list(pairs) + list(false_hypernyms.items())
        self.labels = [1]*len(pairs) + [0]*len(false_hypernyms)
        
    def __getitem__(self, index):
        word, hypernym = self.dataset[index]
        label = self.labels[index]
        
        x = torch.LongTensor([char2int[i] for i in word])
        y = torch.LongTensor([char2int[i] for i in hypernym])
        
        seqlen_x, seqlen_y = torch.LongTensor([len(x)]), torch.LongTensor([len(y)])
        
        fasttext_x = torch.FloatTensor(get_data_vectors(word))
        fasttext_y = torch.FloatTensor(get_data_vectors(hypernym))
        
        return x, fasttext_x, seqlen_x, y, fasttext_y, seqlen_y, label

    def __len__(self):
        return len(self.dataset)

def pad_collate(batch):
    (x, fasttext_x, seqlen_x, y, fasttext_y, seqlen_y, labels) = zip(*batch)
    
    x = pad_sequence(x, batch_first=True, padding_value=0)
    y = pad_sequence(y, batch_first=True, padding_value=0)
    
    fasttext_x = torch.stack(fasttext_x, 0)
    fasttext_y = torch.stack(fasttext_y, 0)

    seqlen_x = torch.stack(seqlen_x, 0).squeeze()
    seqlen_y = torch.stack(seqlen_y, 0).squeeze()

    labels = torch.FloatTensor(labels)
    
    return x, fasttext_x, seqlen_x, y, fasttext_y, seqlen_y, labels

In [30]:
train_iterator = DataLoader(CustomDataset(pairs, false_hypernyms, ftmodel), batch_size=32, shuffle=True, num_workers=0, collate_fn=pad_collate)
val_iterator = DataLoader(CustomDataset(pairs, false_hypernyms, ftmodel), batch_size=32, shuffle=True, num_workers=0, collate_fn=pad_collate)

In [43]:
class CharLSTM(torch.nn.Module):
    def __init__(self, n_tokens, input_size, num_hidden, output_size, fasttext_size):
        super(CharLSTM, self).__init__()
        self.embeddings = nn.Embedding(n_tokens, input_size)
        self.lstm = torch.nn.LSTM(input_size, num_hidden, batch_first=True, bidirectional=True)
        self.relu = torch.nn.ReLU(inplace=True)
        self.dense1 = torch.nn.Linear(fasttext_size*2+4*num_hidden, output_size)
        self.dense2 = torch.nn.Linear(output_size, 1)
        self.dropout = nn.Dropout(p=0.3)
        self.batchnorm = nn.BatchNorm1d(output_size)
        
  
    def forward(self, x, fasttext_x, seqlen_x, y, fasttext_y, seqlen_y):
        x = self.embeddings(x)
        y = self.embeddings(y)
        
        packed_x = pack_padded_sequence(x, seqlen_x, batch_first=True, enforce_sorted=False)
        _, (x, _) = self.lstm(packed_x)
        x = x.permute(1,0,2).reshape(x.shape[1], x.shape[0]*x.shape[2])
        
        packed_y = pack_padded_sequence(y, seqlen_y, batch_first=True, enforce_sorted=False)
        _,(y, _) = self.lstm(packed_y)
        y = y.permute(1,0,2).reshape(y.shape[1], y.shape[0]*y.shape[2])
        
        concatenated = torch.cat((x, fasttext_x, y, fasttext_y), -1)
        x = self.relu(self.dense1(concatenated))
        x = self.batchnorm(x)
        x = self.dropout(x)
        x = self.relu(self.dense2(x))
        return x.squeeze(-1)

In [44]:
model = CharLSTM(50, 256, 512, 128, 300)

In [45]:
x = torch.LongTensor([[13, 15, 24, 23, 9, 32, 24, 7, 26, 5, 29, 30, 9, 13, 24, 15], [13, 15, 24, 23, 9, 32, 24, 7, 26, 5, 29, 30, 9, 13, 24, 15], [13, 15, 24, 23, 9, 32, 24, 7, 26, 5, 29, 30, 9, 13, 24, 15]])
y = torch.LongTensor([[30, 7, 15, 9, 1, 7, 9, 15, 24, 32, 29, 35, 24, 7, 26, 15, 3, 24, 32, 30],[30, 9, 13, 24, 15]+ [0]*15, [30, 9, 13, 24, 15]+ [0]*15])
fasttext_x = torch.FloatTensor(torch.rand((3, 300)))
fasttext_y = torch.FloatTensor(torch.rand((3, 300)))
seqlen_x = torch.LongTensor([x.shape[-1], x.shape[-1], x.shape[-1]])
seqlen_y = torch.LongTensor([20, 5, 5])

In [46]:
model(x, fasttext_x, seqlen_x, y, fasttext_y, seqlen_y)

tensor([0.7782, 0.0000, 0.0000], grad_fn=<SqueezeBackward1>)

In [156]:
def run_epoch(model, iterator, optimizer, criterion, phase='train', epoch=0, writer=None):
    is_train = (phase == 'train')
    if is_train:
        model.train()
    else:
        model.eval()
    
    epoch_loss = 0
    
    with torch.set_grad_enabled(is_train):
        for i, batch in enumerate(iterator):            
            global_i = len(iterator) * epoch + i
            
            # unpack batch
            x, fasttext_x, seqlen_x, y, fasttext_y, seqlen_y, classes = batch
            x, fasttext_x, seqlen_x, y, fasttext_y, seqlen_y = x.cuda(), fasttext_x.cuda(), seqlen_x.cuda(), y.cuda(), fasttext_y.cuda(), seqlen_y.cuda()

            # make prediction
            predictions = model(x, fasttext_x, seqlen_x, y, fasttext_y, seqlen_y)
            answer = torch.round(torch.sigmoid(predictions)).cpu()
            
            
            # calculate loss
            loss = criterion(predictions, classes.cuda())
            
            epoch_loss += loss.item()
            
            if is_train:
                # make optimization step
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
            
            with torch.no_grad():
                # dump train metrics to tensorboard
                if writer is not None and i % 100 == 0 and phase=='train':
                    writer.add_scalar(f"loss/{phase}", loss.item(), global_i)
                if i % 50 == 0:
                    print(torch.sum(answer == classes, dim=-1)/float(answer.shape[0]))
#                     print(predictions)
    
    # dump epoch metrics to tensorboard
    if writer is not None:
        writer.add_scalar(f"loss_epoch/{phase}", epoch_loss / len(iterator), epoch)
                  
    return epoch_loss / len(iterator)

In [157]:
import os
from datetime import datetime
from torch.utils.tensorboard import SummaryWriter

def setup_experiment(title, logdir="./tb"):
    experiment_name = "{}@{}".format(title, datetime.now().strftime("%d-%m-%Y-%H-%M-%S"))
    writer = SummaryWriter(log_dir=os.path.join(logdir, experiment_name))
    best_model_path = f"{title}.baseline.pth"
    return writer, experiment_name, best_model_path

In [158]:
DO_TRAIN = True

In [159]:
if DO_TRAIN:
    model = CharLSTM(len(char2int), 256, 256, 128, 300).cuda()

    optimizer = torch.optim.Adam(model.parameters())
    criterion = nn.BCEWithLogitsLoss()

    writer, experiment_name, best_model_path = setup_experiment(model.__class__.__name__, logdir="./charlstm")
    print(f"Experiment name: {experiment_name}")

    n_epochs = 100
    if not DO_TRAIN:
        n_epochs = 0

    best_val_loss = float('+inf')

    for epoch in tqdm(range(n_epochs)):    
        train_loss = run_epoch(model, train_iterator, optimizer, criterion, phase='train', epoch=epoch, writer=writer)
        val_loss = run_epoch(model, val_iterator, None, criterion, phase='val', epoch=epoch, writer=writer)

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), best_model_path)
        
        print(f'Epoch: {epoch+1:02}')
        print(f'\tTrain Loss: {train_loss:.3f}')
        print(f'\tVal Loss: {val_loss:.3f}')

Experiment name: CharLSTM@23-06-2020-13-55-34







  0%|                                                                                  | 0/100 [00:00<?, ?it/s]

tensor(0.4688)
tensor(0.8125)
tensor(0.8750)
tensor(0.7500)
Epoch: 01
	Train Loss: 0.629
	Val Loss: 0.575







  1%|▋                                                                         | 1/100 [00:03<06:11,  3.75s/it]

tensor(0.7500)
tensor(0.6562)
tensor(0.7188)
tensor(0.8438)
Epoch: 02
	Train Loss: 0.569
	Val Loss: 0.535


...






 15%|██████████▉                                                              | 15/100 [00:54<05:13,  3.69s/it]

tensor(0.8438)
tensor(0.7188)
tensor(0.9688)
tensor(0.9688)
Epoch: 16
	Train Loss: 0.434
	Val Loss: 0.409







 16%|███████████▋                                                             | 16/100 [00:58<05:10,  3.69s/it]

tensor(0.9062)


KeyboardInterrupt: 

In [185]:
model.eval()

CharLSTM(
  (embeddings): Embedding(38, 256)
  (lstm): LSTM(256, 256, batch_first=True, bidirectional=True)
  (relu): ReLU(inplace=True)
  (dense1): Linear(in_features=1624, out_features=128, bias=True)
  (dense2): Linear(in_features=128, out_features=1, bias=True)
  (dropout): Dropout(p=0.3, inplace=False)
  (batchnorm): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)

In [214]:
word = "ganache"
hypernym_candidate = "meat"
x, y  = [char2int[i] for i in word], [char2int[i] for i in hypernym_candidate]
seqlen_x, seqlen_y = len(x), len(y)
fasttext_x = torch.FloatTensor([get_data_vectors(word)])#.unsqueeze(0)
fasttext_y = torch.FloatTensor([get_data_vectors(hypernym_candidate)])#.unsqueeze(0)

In [215]:
x = torch.LongTensor([x])
y = torch.LongTensor([y])
seqlen_x, seqlen_y = torch.LongTensor([seqlen_x]),  torch.LongTensor([seqlen_y])

x.shape, y.shape, seqlen_x.shape, seqlen_y.shape, fasttext_x.shape, fasttext_y.shape 

(torch.Size([1, 7]),
 torch.Size([1, 4]),
 torch.Size([1]),
 torch.Size([1]),
 torch.Size([1, 300]),
 torch.Size([1, 300]))

In [216]:
torch.round(torch.sigmoid(model(x.cuda(), fasttext_x.cuda(), seqlen_x.cuda(), y.cuda(), fasttext_y.cuda(), seqlen_y.cuda())))

tensor([0.], device='cuda:0', grad_fn=<RoundBackward>)

In [182]:
for i, batch in enumerate(train_iterator):
            
    # unpack batch
    x, fasttext_x, seqlen_x, y, fasttext_y, seqlen_y, classes = batch
    x, fasttext_x, seqlen_x, y, fasttext_y, seqlen_y = x.cuda(), fasttext_x.cuda(), seqlen_x.cuda(), y.cuda(), fasttext_y.cuda(), seqlen_y.cuda()

    # make prediction
    predictions = model(x, fasttext_x, seqlen_x, y, fasttext_y, seqlen_y)
    print(predictions, classes)
    print(criterion(predictions, classes.cuda()).item())

tensor([ 0.0000,  7.8896,  0.5138,  3.1322,  0.0000,  5.3210, 12.8890,  3.7901,
         6.0119,  0.0000,  0.0000,  0.0000,  0.0000,  6.7213,  5.0626,  0.0000,
         0.0000,  4.3142,  0.0000,  0.0000,  0.0000,  0.0000,  4.1570,  0.0000,
         0.0000,  0.0000,  0.0000,  7.0024,  6.0456,  0.0000,  0.0000,  2.0749],
       device='cuda:0', grad_fn=<SqueezeBackward1>) tensor([0., 1., 1., 1., 0., 1., 1., 1., 1., 0., 0., 0., 0., 1., 1., 1., 0., 1.,
        0., 1., 0., 0., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1.])
0.4117552638053894
tensor([ 0.0000,  1.7270,  0.0000,  0.0000,  9.7684,  0.0000,  7.7403,  0.0000,
         0.0000, 11.1195,  5.4869,  4.3389,  0.0000,  9.9367,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  3.9408,  0.0000,  0.0000,  0.0000,  0.0000,
         4.6579,  0.0000,  0.0000,  0.0000,  0.0000,  5.0753,  7.8502,  6.4034],
       device='cuda:0', grad_fn=<SqueezeBackward1>) tensor([0., 1., 1., 0., 1., 0., 1., 0., 0., 1., 1., 1., 1., 1., 0., 1., 0., 0.,
        0.

       grad_fn=<SqueezeBackward1>) tensor([0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 0., 0., 0.,
        1., 0., 1., 0., 1., 1., 1., 1., 0., 0., 0., 1., 0., 1.])
0.36026936769485474
tensor([5.3532, 0.0000, 0.0000, 0.0000, 9.6597, 0.0000, 0.0000, 0.7741, 7.2072,
        0.0000, 0.0000, 0.0000, 0.0000, 6.5106, 0.0000, 3.8386, 8.5620, 0.0000,
        3.4275, 7.4604, 3.8053, 1.1464, 0.0000, 0.0000, 0.0000, 0.0000, 6.9514,
        0.0000, 0.0000, 3.4100, 0.0000, 0.0000], device='cuda:0',
       grad_fn=<SqueezeBackward1>) tensor([1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 1., 0., 0., 1., 0., 1., 1., 0.,
        1., 1., 1., 1., 0., 1., 0., 0., 1., 0., 0., 1., 1., 0.])
0.4598608613014221
tensor([2.1319, 6.2035, 3.3787, 1.6171, 0.0000, 0.0000, 5.0459, 0.2965, 0.0000,
        3.4802, 1.2367, 3.9181, 0.0000, 3.0364, 0.0000, 4.7740, 6.7876, 0.0000,
        0.0000, 0.0000, 3.4946, 7.5116, 8.9196, 0.0000, 0.0000, 2.3085, 0.0000,
        5.6131, 0.0000, 0.0000, 0.7274, 0.0000], device

       grad_fn=<SqueezeBackward1>) tensor([1., 1., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 1., 0., 0., 1., 0., 1.,
        1., 0., 1., 0., 1., 0., 1., 0., 1., 1., 0., 0., 0., 1.])
0.4713408350944519
tensor([ 0.0000,  0.0000,  0.0000,  0.5646,  0.0000,  5.5455,  8.7503,  8.9479,
         0.0000,  0.4511,  0.0000,  0.0000,  0.0000,  0.0000, 13.1739,  7.8363,
         0.0000,  0.0000,  2.6981,  0.0000,  0.0000,  0.0000,  6.3981, 10.0460,
         0.0000,  0.0000,  4.1890,  0.0000,  0.0000,  0.0000,  7.7575,  0.0000],
       device='cuda:0', grad_fn=<SqueezeBackward1>) tensor([0., 0., 1., 0., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 1., 1., 1., 0.,
        1., 0., 0., 0., 1., 1., 0., 0., 1., 0., 0., 0., 1., 0.])
0.4971431493759155
tensor([6.7782, 3.8308, 0.0000, 2.9512, 0.0000, 0.0000, 0.0000, 0.0000, 4.7506,
        0.0000, 4.3639, 0.0000, 0.0000, 9.2395, 0.0000, 9.4509, 0.0000, 6.6414,
        0.3115, 0.0000, 0.0000, 5.9664, 6.2528, 6.3587, 0.0000, 0.0000, 0.0000,
        0.0000, 4.1509, 

       device='cuda:0', grad_fn=<SqueezeBackward1>) tensor([0., 1., 1., 0., 1., 0., 1., 1., 1., 0., 1., 1., 0., 0., 1., 0., 1., 1.,
        0., 0., 1., 0., 1., 0., 1., 1., 1., 0., 1., 0., 1., 0.])
0.3830859363079071
tensor([ 0.0000,  0.0000,  0.0000,  5.6154, 10.5985,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  5.0016,  0.5551,  0.0000,  0.0000,  6.7029,  0.0000,
         6.8591,  0.0000,  0.0000,  0.0000,  6.6081,  0.0000,  6.9750, 11.0999,
         0.0000,  0.9349,  2.4612,  0.0000,  0.0000,  0.0000,  4.6530,  6.3166],
       device='cuda:0', grad_fn=<SqueezeBackward1>) tensor([0., 0., 1., 1., 1., 0., 0., 0., 0., 0., 1., 1., 0., 0., 1., 0., 1., 0.,
        0., 1., 1., 0., 1., 1., 0., 1., 0., 0., 1., 0., 1., 1.])
0.5163717865943909
tensor([2.3151, 3.4891, 0.0000, 0.0000, 0.0000, 0.0000, 3.2696, 0.0000, 0.0000,
        1.1480, 3.4280, 5.3745, 0.7041, 3.8876, 4.8723, 6.2164, 0.0000, 0.0000,
        2.2322, 2.5883, 6.0915, 0.0000, 0.0000, 1.3463, 6.5081, 1.6448, 1.9544,
       

       grad_fn=<SqueezeBackward1>) tensor([0., 1., 1., 0., 0., 0., 0., 1., 1., 0., 1., 1., 0., 0., 0., 1., 1., 0.,
        1., 1., 0., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 0.])
0.3144753575325012
tensor([0.0000, 2.8224, 3.9671, 2.6501, 5.1974, 0.0000, 6.5688, 0.0000, 0.0000,
        0.0000, 1.9747, 0.0000, 0.0000, 6.2320, 2.9256, 8.7969, 4.4011, 4.0630,
        2.9239, 0.0000, 3.7133, 2.4078, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.8790, 0.0000, 5.1616, 4.8505, 1.0682], device='cuda:0',
       grad_fn=<SqueezeBackward1>) tensor([0., 1., 1., 1., 1., 0., 1., 0., 1., 1., 1., 0., 0., 1., 1., 1., 1., 1.,
        1., 0., 1., 1., 0., 1., 0., 0., 0., 1., 0., 1., 1., 1.])
0.3402531147003174
tensor([ 0.0000,  0.0000,  3.6451,  0.0000,  0.0000,  0.0000,  6.7622,  7.4946,
         0.0000,  8.8281,  0.0000, 11.9047,  2.1142,  0.0000,  0.0000,  7.7349,
         0.0000,  0.0000,  0.0000,  0.0000,  1.7860,  0.0000,  6.3141,  0.0000,
         0.0000,  3.3701,  0.0000,  0.0000,  5.7266,  0.

       grad_fn=<SqueezeBackward1>) tensor([0., 0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 1.,
        1., 0., 0., 0., 0., 1., 0., 0., 1., 1., 0., 1., 1., 0.])
0.41521206498146057
tensor([4.0522, 0.0000, 0.0000, 2.2498, 2.8745, 0.0000, 4.9168, 0.0000, 5.2306,
        7.0365, 0.0000, 0.0000, 0.0000, 5.6177, 7.3123, 1.8557, 0.0000, 0.0000,
        7.2953, 3.8030, 0.0000, 0.0000, 1.1540, 0.0000, 2.4619, 0.0000, 8.1052,
        6.2865, 8.1326, 0.0000, 0.0000, 0.0000], device='cuda:0',
       grad_fn=<SqueezeBackward1>) tensor([1., 0., 0., 1., 1., 0., 1., 0., 1., 1., 0., 0., 0., 1., 1., 1., 1., 0.,
        1., 1., 0., 1., 1., 0., 1., 1., 1., 1., 1., 0., 1., 0.])
0.3689664900302887
tensor([8.6612, 0.0000, 4.2769, 0.0000, 5.5932, 0.0000, 6.1233, 0.0000, 0.0000,
        0.0000, 0.0000, 7.6719, 0.0000, 0.3004, 6.2172, 3.6806, 0.0000, 0.0000,
        0.0000, 0.0000, 5.7484, 3.3584, 0.0000, 0.0000, 6.9436, 0.0000, 7.2089,
        7.2441, 2.3417, 0.0000, 0.0000, 6.3646], device